In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,167 kB]
Hit:14 http://ppa.lau

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-30 17:22:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2022-04-30 17:22:53 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession

In [5]:
spark_engine = SparkSession.builder.appName("bigdata_level_1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
# Load Amazon Data into Spark DataFrame

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark_engine.sparkContext.addFile(url)
amazon_reviews_df= spark_engine.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [8]:
# Filter by votes
filter_dataset = amazon_reviews_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [11]:
# Filter for greater than 20 total votes
filter_dataset= filter_dataset.filter(filter_dataset["total_votes"]>20)
filter_dataset.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           25|         26|   N|                Y|
|          5|           25|         26|   N|                N|
|          5|           19|         21|   N|                Y|
|          5|           46|         48|   N|                N|
|          4|           46|         46|   N|                N|
|          5|          292|        300|   N|                N|
|          4|           21|         22|   N|                N|
|          5|           26|         26|   N|                N|
|          2|           11|         22|   N|                N|
|          5|           26|         30|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [13]:
# # Filter for greater than 50% of helpful votes
helpful_percentage_df = filter_dataset.filter(filter_dataset["helpful_votes"]/filter_dataset["total_votes"]>=0.5)
helpful_percentage_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           25|         26|   N|                Y|
|          5|           25|         26|   N|                N|
|          5|           19|         21|   N|                Y|
|          5|           46|         48|   N|                N|
|          4|           46|         46|   N|                N|
|          5|          292|        300|   N|                N|
|          4|           21|         22|   N|                N|
|          5|           26|         26|   N|                N|
|          2|           11|         22|   N|                N|
|          5|           26|         30|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [14]:
# Descriptive statistics for paid reviews
vine_df = helpful_percentage_df.filter(helpful_percentage_df["vine"]== 'Y')
vine_df.describe().show(10)

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|                 7|                 7|                 7|   7|                7|
|   mean|3.5714285714285716|26.857142857142858|31.714285714285715|null|             null|
| stddev|0.5345224838248488|13.582201238245238|14.032275720807439|null|             null|
|    min|                 3|                15|                21|   Y|                N|
|    max|                 4|                54|                58|   Y|                N|
+-------+------------------+------------------+------------------+----+-----------------+



In [15]:
# Descriptive statistics for unpaid reviews
vine_df_n = helpful_percentage_df.filter(helpful_percentage_df["vine"]== 'N')
vine_df_n.describe().show(10)

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             96798|             96798|             96798|96798|            96798|
|   mean| 4.202493853178785| 36.11625240190913| 40.40868612987872| null|             null|
| stddev|1.2867927548410474|37.094365757438574|39.728175223837276| null|             null|
|    min|                 1|                11|                21|    N|                N|
|    max|                 5|              2181|              2246|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+

